In [1]:
import pandas as pd

In [3]:
hr_data = pd.read_csv('data/HR_comma_sep.csv')

In [4]:
hr_data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [6]:
hr_data.rename(columns={'sales':'department'},inplace=True)

In [8]:
hr_data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
le = LabelEncoder()

In [12]:
hr_data.department = le.fit_transform(hr_data.department)

In [13]:
hr_data.salary = le.fit_transform(hr_data.salary)

In [15]:
hr_data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,7,1
1,0.80,0.86,5,262,6,0,1,0,7,2
2,0.11,0.88,7,272,4,0,1,0,7,2
3,0.72,0.87,5,223,5,0,1,0,7,1
4,0.37,0.52,2,159,3,0,1,0,7,1


In [17]:
#list comprehension - find all the colns except left
feature_col = [ col for col in hr_data.columns.tolist() if col != 'left']

In [19]:
feature_data = hr_data[feature_col]

In [20]:
target_data = hr_data['left']

In [22]:
len(feature_data.columns.tolist())

9

In [23]:
import tflearn
import tensorflow as tf

In [24]:
l = [1,2,3]
list(enumerate(l))

[(0, 1), (1, 2), (2, 3)]

In [25]:
def dnn_model(num_layers,layer_info):
    tf.reset_default_graph()
    
    #create units for input layer
    net = tflearn.input_data(shape=[None,9])
    
    #create hidden layers
    for idx,layer in enumerate(range(1,num_layers+1)):
        net = tflearn.fully_connected(net,layer_info[idx],activation='ReLU')
    
    #create output layer
    net = tflearn.fully_connected(net,2,activation='softmax')
    net = tflearn.regression(net,optimizer='sgd',learning_rate=0.01, loss='categorical_crossentropy')
    
    model = tflearn.DNN(net)
    return model

In [26]:
model = dnn_model(2,[64,32])

In [27]:
from sklearn.preprocessing import OneHotEncoder

In [28]:
ohe = OneHotEncoder()

In [32]:
trainY = ohe.fit_transform(target_data.values.reshape(-1,1))

In [34]:
#Convert sparse matrix to dense matrix
trainY.toarray()

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [44]:
#Convert pandas into numpy
feature_data.as_matrix()

array([[ 0.38,  0.53,  2.  , ...,  0.  ,  7.  ,  1.  ],
       [ 0.8 ,  0.86,  5.  , ...,  0.  ,  7.  ,  2.  ],
       [ 0.11,  0.88,  7.  , ...,  0.  ,  7.  ,  2.  ],
       ..., 
       [ 0.37,  0.53,  2.  , ...,  0.  ,  8.  ,  1.  ],
       [ 0.11,  0.96,  6.  , ...,  0.  ,  8.  ,  1.  ],
       [ 0.37,  0.52,  2.  , ...,  0.  ,  8.  ,  1.  ]])

In [37]:
model.fit(feature_data.as_matrix(),trainY.toarray(),validation_set=0.01,show_metric=True, batch_size=100,n_epoch=50)

Training Step: 7449  | total loss: 0.53828 | time: 0.337s
| SGD | epoch: 050 | loss: 0.53828 - acc: 0.7690 -- iter: 14800/14849
Training Step: 7450  | total loss: 0.53837 | time: 1.347s
| SGD | epoch: 050 | loss: 0.53837 - acc: 0.7691 | val_loss: 0.57430 - val_acc: 0.7400 -- iter: 14849/14849
--


In [38]:
model = dnn_model(2,[8,4])

In [39]:
model.fit(feature_data.as_matrix(),trainY.toarray(),validation_set=0.01,show_metric=True, batch_size=100,n_epoch=50)

Training Step: 7449  | total loss: 0.53828 | time: 0.332s
| SGD | epoch: 050 | loss: 0.53828 - acc: 0.7710 -- iter: 14800/14849
Training Step: 7450  | total loss: 0.54187 | time: 1.344s
| SGD | epoch: 050 | loss: 0.54187 - acc: 0.7679 | val_loss: 0.51991 - val_acc: 0.7867 -- iter: 14849/14849
--


In [40]:
model = dnn_model(3,[16,8,4])

In [41]:
model.fit(feature_data.as_matrix(),trainY.toarray(),validation_set=0.01,show_metric=True, batch_size=100,n_epoch=50)

Training Step: 7449  | total loss: 0.55446 | time: 0.314s
| SGD | epoch: 050 | loss: 0.55446 - acc: 0.7571 -- iter: 14800/14849
Training Step: 7450  | total loss: 0.55062 | time: 1.321s
| SGD | epoch: 050 | loss: 0.55062 - acc: 0.7604 | val_loss: 0.51217 - val_acc: 0.7933 -- iter: 14849/14849
--


In [56]:
res = model.predict_label(feature_data.as_matrix()[0:4])

In [59]:
ohe.active_features_

array([0, 1])

In [61]:
a = pd.Series(data=[5,6,7,6])

In [62]:
a.values

array([5, 6, 7, 6])

In [63]:
ohee = OneHotEncoder()

In [66]:
ohee.fit_transform(a.values.reshape(-1,1)).toarray()

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.]])

In [67]:
ohee.active_features_

array([5, 6, 7])